In [ ]:
import sys
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'mapie',
    'causal',
    'wkmeans',
]

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='halving',
        train_start=datetime(2018,4,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='random', search_type='wkmeans', search_subtype='wasserstein',
        n_models=10, n_jobs=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None
if not study:
    n_trials = 500
else:
    n_trials = len(study.trials) + 500

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 20/519 ins=-1.000000 oos=-1.000000 avg=5.67s mem=933.73MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 21/519 ins=-1.000000 oos=-1.000000 avg=5.79s mem=1178.57MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 22/519 ins=0.092173 oos=0.033735 avg=6.60s mem=1176.57MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 23/519 ins=0.905890 oos=0.317526 avg=6.52s mem=1211.29MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 24/519 ins=0.905890 oos=0.317526 avg=6.58s mem=1216.04MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 25/519 ins=0.905890 oos=0.317526 avg=6.70s mem=1100.74MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 26/519 ins=0.905890 oos=0.317526 avg=6.55s mem=1100.96MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 27/519 ins=0.905890 oos=0.317526 avg=6.32s mem=1112.34MB 
[XAUUSD_H1_buy_random_wkmeans_wasserstein] modelo 0 trial 28/519 ins=